In [1]:
import urllib.request
import requests
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import re
import time
from tqdm import tqdm
import pandas as pd
from collections import OrderedDict

In [13]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

In [49]:
def find_bloggers(username, password, topic):

    driver = webdriver.Firefox()
    driver.get('http://weibo.com/login.php')
    time.sleep(5)
    driver.find_element_by_name('username').send_keys(username)
    driver.find_element_by_name('password').send_keys(password)
    time.sleep(10) # time to input validation code
    
    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/div/div/div/div[3]/div[6]').click()
    time.sleep(20)
    

    driver.get('https://s.weibo.com/')
    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/a[2]').click()
    time.sleep(5)

    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').clear()
    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').send_keys(topic)
    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/button').click()
    time.sleep(5)

    elem = driver.find_element_by_link_text('标签').click()
    time.sleep(2)
    ele = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[1]/ul/li[2]/span')
    ActionChains(driver).move_to_element(ele).perform()
    time.sleep(1)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[1]/ul/li[2]/span/ul/li[3]/a').click()
    time.sleep(5)

    data = {}

    for j in range(1, 50):
        try:

            for i in range(len(driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div[1]/div[3]/*'))):

                table = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[3]/div['+str(i+1)+']/div[2]')

                if table.find_element_by_xpath('p[3]').text[:2] != '关注':
                    continue

                name = table.find_element_by_xpath('div/a[1]').text    
                uid = table.find_element_by_xpath('p[3]/span[1]/a').get_attribute('href')
                uid = re.findall('[0-9]+', uid)[0]
                gender = table.find_element_by_xpath('p[1]/i').get_attribute('class')
                gender = re.findall('icon-sex-(.*)', gender)[0]
                region = table.find_element_by_xpath('p[1]').text.split(' ')[0]

                if len(table.find_elements_by_xpath('p[2]/*')) == 0:
                    tag = table.find_element_by_xpath('p[2]').text.split(' ')
                    follows = table.find_element_by_xpath('p[3]/span/a').text
                    fans = table.find_element_by_xpath('p[3]/span[2]/a').text
                    posts = table.find_element_by_xpath('p[3]/span[3]/a').text
                else:
                    tag = []
                    follows = table.find_element_by_xpath('p[2]/span/a').text
                    fans = table.find_element_by_xpath('p[2]/span[2]/a').text
                    posts = table.find_element_by_xpath('p[2]/span[3]/a').text

                data[len(data)] = {'name':name, 'uid':uid, 'gender':gender,'region':region,
                                   'tag':tag,'follows':follows,'fans':fans,'posts':posts}

            time.sleep(5)

            if j != 43:
                driver.find_element_by_class_name('next').click()

            time.sleep(5)
        except:
            break
    return data

In [50]:
def dict_to_df(data):
    
    df = pd.DataFrame(data).T
    cols = ['name', 'uid', 'gender', 'region', 'tag', 'follows', 'fans', 'posts']
    df = df.loc[:,cols]
    
    for i in range(len(df)):
        
        df['tag'][i] = ', '.join(df['tag'][i])
        
        if bool(re.search(u'[\u4e00-\u9fa5]+', df['follows'][i])):
            df['follows'][i] = df['follows'][i][:-1] + '0000'
        else:
             df['follows'][i] = df['follows'][i]
        
        if bool(re.search(u'[\u4e00-\u9fa5]+', df['fans'][i])):
            df['fans'][i] = df['fans'][i][:-1] + '0000'
        else:
             df['fans'][i] = df['fans'][i]
                
        if bool(re.search(u'[\u4e00-\u9fa5]+', df['posts'][i])):
            df['posts'][i] = df['posts'][i][:-1] + '0000'
        else:
             df['posts'][i] = df['posts'][i]
                
    return df

In [51]:
brand_name = ['凯迪拉克', '奥迪', '雷克萨斯', '宝马', '奔驰', '沃尔沃']

In [52]:
data1 = find_bloggers('0402_85e736@163.com', "cpy919992", "凯迪拉克")
df1 = dict_to_df(data1)
df1

,name,uid,gender,region,tag,follows,fans,posts
0,38号美系性能控,2017038601,male,北京,"独立车评人, 微博签约自媒体",1534,3130000,10000
1,女司机开车了,2596102331,female,甘肃,汽车博主,1691,720000,3407
2,袋鼠国的泰勒蜀黍,3038862623,male,海外,"教育博主, 教育培训答主",316,510000,2412
3,汽车人八卦,1915129857,male,浙江,浙江米卡迪汽车销售服务有限公司总经理张建业,3228,60000,30000
4,福莱西宝,1731924557,male,上海,汽车博主,1421,5817,10000
5,中国需要成吉思汗,2662574693,male,河南,北京宏源洁保洁服务有限责任公司总经理,1391,80000,20000
6,二手车评估师周思远,3698264287,male,辽宁,"沈阳市北方汽修专业学校教师, 汽车博主",312,10000,814
7,圣西罗的红与黑,1748930212,male,山东,东营灿林商贸有限公司总经理纪涛,633,6117,10000
8,八婺饭堂,1215680047,male,浙江,摄影博主,194,3038,1091
9,居凯,17993288,male,江苏,"半渡莲国粹文化公馆, 创办人",3109,2323,1124


In [53]:
df1.to_csv('凯迪拉克.csv',  encoding = 'utf_8_sig')

In [54]:
data2 = find_bloggers('0402_85e736@163.com', "cpy919992", "奥迪")
df2 = dict_to_df(data2)
df2
df2.to_csv('奥迪.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag,follows,fans,posts
0,确实是个好BOSS,1853981457,male,上海,"微博美食博主, 美食视频自媒体",999,220000,6711
1,汽车全搜罗,1784830527,male,北京,"知名汽车博主, 汽车达人",233,2480000,7790
2,全球好车榜,1409477073,female,北京,资深汽车达人,395,80000,10000
3,红儿宝贝美媚,3972964551,male,广东,"人文艺术博主, 头条文章作者, 微博签约自媒体",131,2020000,7402
4,孙大诚,1237630473,male,山东,"汽车之家超级试驾员, 汽车博主, 微博签约自媒体",779,290000,6691
...,...,...,...,...,...,...,...,...
201,刘禾Even,2651459590,male,四川,"成都华星名仕汽车销售服务有限公司, 销售培训经理",347,139,48
202,夏传华,3525564693,male,安徽,"合肥鑫诚轿车技术服务有限公司, 总经理",98,105,122
203,李越的官博,2831961990,male,上海,"上海勇哈实业有限公司, 研发部产品设计与开发总监",42,121,51
204,德系修车技师陆师傅,6011422526,male,浙江,义乌市奔宝汽车修理有限公司高级维修技师,155,117,35


In [55]:
data3 = find_bloggers('0402_85e736@163.com', "cpy919992", "雷克萨斯")
df3 = dict_to_df(data3)
df3
df3.to_csv('雷克萨斯.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag,follows,fans,posts
0,Cars车资讯,1661672197,female,重庆,汽车博主,162,1460000,4059
1,deepsoul_,1408629490,male,北京,FAST4WARD创始人,496,40000,10000
2,Yuan_Wayne,1937867213,male,江苏,"摄影师, Adobe中国认证设计师",335,6341,8
3,语彤Karen,1610171522,female,北京,"商务主持人孟婧，现担任国际车展, 丰田汽车、雷克萨斯御用主持人, 运动视频自媒体",535,40000,243
4,肖----杰,1981021041,male,辽宁,"重庆中升雷克萨斯汽车销售服务有限公司, 总经理",857,20000,2931
5,美东雷克萨斯售后老贺,2088666101,male,福建,厦门美东汽车销售服务有限公司售后经理,195,8130,1613
6,开心凌志,1904731695,female,北京,北京花园桥雷克萨斯4S店总经理,250,2990,2282
7,盛元珺,1889559700,male,上海,"汽车之家上海分站, 责任编辑",748,3055,3951
8,张京333,1883614152,male,北京,安徽铂丽房地产投资有限公司执行董事总经理张京,152,2737,185
9,sky---飞,1737481233,male,上海,"上海英之杰雷克萨斯, 市场专员",407,2385,68


In [56]:
data4 = find_bloggers('0402_85e736@163.com', "cpy919992", "宝马")
df4 = dict_to_df(data4)
df4
df4.to_csv('宝马.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag,follows,fans,posts
0,跑车世界,2015944007,male,上海,"资深汽车达人, 汽车视频自媒体",2875,3850000,50000
1,全球汽车阿飞说,1757599201,male,北京,"知名汽车博主, 《名车》评论撰稿人, 全球汽车超话主持人, 汽车视频自媒体",803,6090000,10000
2,李克LIKEHOOP,1648844612,male,北京,微博签约自媒体,968,2250000,20000
3,时尚潮流汽车,1769771933,female,上海,"知名汽车博主, 汽车达人",350,3040000,6185
4,汽车全搜罗,1784830527,male,北京,"知名汽车博主, 汽车达人",233,2480000,7790
...,...,...,...,...,...,...,...,...
231,夏传华,3525564693,male,安徽,"合肥鑫诚轿车技术服务有限公司, 总经理",98,105,122
232,李越的官博,2831961990,male,上海,"上海勇哈实业有限公司, 研发部产品设计与开发总监",42,121,51
233,德系修车技师陆师傅,6011422526,male,浙江,义乌市奔宝汽车修理有限公司高级维修技师,155,117,35
234,高亮-Limited,5778824415,male,天津,头条文章作者,22,17,267


In [58]:
data5 = find_bloggers('0402_85e736@163.com', "cpy919992", "奔驰")
df5 = dict_to_df(data5)
df5
df5.to_csv('奔驰.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag,follows,fans,posts
0,跑车世界,2015944007,male,上海,"资深汽车达人, 汽车视频自媒体",2875,3850000,50000
1,全球汽车阿飞说,1757599201,male,北京,"知名汽车博主, 《名车》评论撰稿人, 全球汽车超话主持人, 汽车视频自媒体",803,6090000,10000
2,白宁的爱车时光,1267250293,male,北京,"头条文章作者, 微博签约自媒体",1960,830000,8628
3,汽车时尚馆,1879429172,female,北京,资深汽车达人,334,850000,10000
4,时尚潮流汽车,1769771933,female,上海,"知名汽车博主, 汽车达人",350,3040000,6185
...,...,...,...,...,...,...,...,...
200,德系修车技师陆师傅,6011422526,male,浙江,义乌市奔宝汽车修理有限公司高级维修技师,155,117,35
201,艺哥,5908941326,male,北京,奔驰汽车销售有限公司执行总监,77,99,157
202,高亮-Limited,5778824415,male,天津,头条文章作者,22,17,267
203,小小小达i,6035491364,male,河北,微博原创视频博主,119,10000,30


In [60]:
data6 = find_bloggers('0402_85e736@163.com', "cpy919992", "沃尔沃")
df6 = dict_to_df(data6)
df6
df6.to_csv('沃尔沃.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag,follows,fans,posts
0,女司机开车了,2596102331,female,甘肃,汽车博主,1691,720000,3407
1,刘圣伟_RosenWide,36585545,male,陕西,"西安华道生物科技有限公司, 网络部总监",76,20000,1662
2,Ly-nan,1601015355,male,福建,福建省武夷山维创设计—执行总裁,259,1753,20
3,媚儿_golf,1298183683,female,四川,沃尔沃中国公开赛媒体经理;北京九州盛世文化传播成都首席代表,422,8595,3579
4,FionaLam_Volvocars,2527969212,female,上海,沃尔沃汽车销售有限公司-营销传播总监林怡,186,5934,190
5,芒果稀饭,1780671353,male,贵州,"贵州天鼎沃尔沃汽车市场总监, 姚嘉",742,5735,3899
6,土豆木公,1456023377,male,重庆,媒体人,1326,2813,2094
7,箱子singing,1874552567,male,上海,宜沃数字科技总经理,1642,2272,5392
8,李雨航Ril,1169633952,male,陕西,陕西力鹄商贸有限公司技术部主管,111,1636,872
9,单口相声Gemini,1852076047,male,云南,昆明理工大学城市学院团委副书记,804,1457,10000


In [222]:
def find_bloggers2(username, password, topic):
    driver = webdriver.Firefox()
    driver.get('http://weibo.com/login.php')
    time.sleep(5)
    driver.find_element_by_name('username').send_keys(username)
    driver.find_element_by_name('password').send_keys(password)
    time.sleep(10) # time to input validation code

    driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/div/div/div/div[3]/div[6]').click()
    time.sleep(5)


    driver.get('https://s.weibo.com/')
    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/a[2]').click()
    time.sleep(5)

    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').clear()
    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').send_keys(topic)
    driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/button').click()
    time.sleep(5)

    elem = driver.find_element_by_link_text('标签').click()
    time.sleep(2)
    ele = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[1]/ul/li[2]/span')
    ActionChains(driver).move_to_element(ele).perform()
    time.sleep(1)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[1]/ul/li[2]/span/ul/li[4]/a').click()
    time.sleep(5)

    data = {}
    for j in range(1, 30):
        try:
            s = driver.find_element_by_class_name('card.card-no-result.s-pt20b40')
            driver.refresh()
            time.sleep(5)
        except:
            time.sleep(1)

        for i in range(len(driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div[1]/div[3]/*'))):

            table = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[3]/div['+str(i+1)+']/div[2]')

            if table.find_element_by_xpath('p[3]').text[:2] == '关注':
                continue

            name = table.find_element_by_xpath('div/a[1]').text    
            uid = table.find_element_by_class_name('name').get_attribute('href')
            gender = table.find_element_by_xpath('p[1]/i').get_attribute('class')
            gender = re.findall('icon-sex-(.*)', gender)[0]
            region = table.find_element_by_xpath('p[1]').text.split(' ')[0]
            tags = []
            for ele in table.find_elements_by_xpath('p'):
                if ele.text.startswith('标签'):
                    tag_lst = ele.find_elements_by_xpath('a')
                    tags = [tag.text for tag in tag_lst]

            data[len(data)] = {'name':name, 'uid':uid, 'gender':gender,'region':region,
                               'tag': tags}
        time.sleep(5)

        try:
            driver.find_element_by_class_name('next').click()
            time.sleep(4)
        except:
            break
    return data

In [217]:
def dict_to_df2(data):
    
    df = pd.DataFrame(data).T
    cols = ['name', 'uid', 'gender', 'region', 'tag']
    df = df.loc[:,cols]
    
    for i in range(len(df)):
        
        df['tag'][i] = ', '.join(df['tag'][i])
                
    return df

In [218]:
data1 = find_bloggers2('0402_85e736@163.com', "cpy919992", "凯迪拉克")
df1 = dict_to_df2(data1)
df1
df1.to_csv('凯迪拉克_o.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag
0,潮流风景线,https://weibo.com/u/2700753641,male,河南,"闺女徐贤要幸福, 罗塘人家, 爱情小白, 凯迪拉克SRX"
1,扣_扣_扣扣篮,https://weibo.com/szeyuen1008,male,广东,"英语, 篮球, 爱狗, 衛蘭, 周傑倫, 保时捷, 凯迪拉克, 别克, 雷克萨斯, 起亚"
2,凯迪拉克长春宏明品鉴中心,https://weibo.com/726260820,male,吉林,"凯迪拉克, 促销, 管理, 销售, 营销, 市场, 百年历史, 美国品牌, 豪华车, 汽车"
3,北巷草猫,https://weibo.com/libamao,male,江苏,"IT数码, 军事, 旅游, 新闻资讯, 汽车, 打球, 凯迪拉克"
4,凯迪拉克大本营,https://weibo.com/u/2375199402,male,广东,"凯迪拉克, CTS, 轿跑, 钻石切割, SLS, 凯雷德, 豪华车, SRX, 汽车, 赛威"
...,...,...,...,...,...
536,心若向阳16觉醒,https://weibo.com/345983188,male,广东,"福特, 通用, 拆车件, 凯迪拉克, 汽配, 体育, 篮球, 武动乾坤"
537,18岁的家馨sweetheart,https://weibo.com/u/1781805214,female,广东,凯迪拉克
538,18岁的博明Carina,https://weibo.com/u/1778596120,male,吉林,凯迪拉克
539,18岁的博延pumpkin,https://weibo.com/u/1778571130,female,台湾,凯迪拉克


In [219]:
data2 = find_bloggers2('0402_85e736@163.com', "cpy919992", "奥迪")
df2 = dict_to_df2(data2)
df2
df2.to_csv('奥迪_o.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag
0,全球豪车馆,https://weibo.com/u/2335798650,male,云南,"汽车, 豪车, 名车, 跑车, 全球豪车馆, 宝马, 奔驰, 奥迪, 兰博基尼, 法拉利"
1,德国超哥,https://weibo.com/longzhichao,male,海外,"德国, 电动汽车, 旅游爱好者, 德国汽车, 保时捷, 海拉, 奥迪, 库卡"
2,零号ME,https://weibo.com/michael2046,male,广东,"兰博基尼, 法拉利, 保时捷, 深圳车友, 劳斯莱斯, 豪车保养, 改装, 奔驰宝马奥迪, ..."
3,蓬蓬Perry,https://weibo.com/perrywpy,male,上海,"KPMG, 听歌, 看美剧, 奥迪特"
4,东莞奥丽汽车专业升级,https://weibo.com/dgaoli,male,广东,"改装, 汽车, 奥丽, 宝马, 奔驰, 大众, 奥迪, 丰田, 排气, 避震"
...,...,...,...,...,...
547,文龙mourinho,https://weibo.com/u/1700075785,male,广东,"怀旧, 车仔, 美食, 奥迪, 太极, 诗人, 酒, 广州, 愤青, 茶客"
548,帅哥不洗脚,https://weibo.com/mikastephen,male,浙江,"斯诺克, 奥迪控, 自驾游, 穷人囧"
549,绝地武士的马仔,https://weibo.com/peterq187155433,male,广东,"机场, 飞机, 奥迪, 德系汽车, 好莱坞大片, 东北偏东"
550,只爱言小君,https://weibo.com/u/1727730265,female,广东,"为了你的奥迪, 我的迪奥, 奋斗吧～"


In [224]:
data3 = find_bloggers2('0402_85e736@163.com', "cpy919992", "雷克萨斯")
df3 = dict_to_df2(data3)
df3
df3.to_csv('雷克萨斯_o.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag
0,沈忱忱忱忱,https://weibo.com/u/1791670113,female,北京,"天蝎座, 雷克萨斯, 音乐, 宅, 学生, 星座命理"
1,周雷-LEXUS,https://weibo.com/zhoulei302,male,北京,"市场营销, 市场分析, 策划, 市场推广, 媒体公关, 雷克萨斯, 北京, 6年行业经验"
2,贵阳爱都沪美人微整形医院,https://weibo.com/u/5042969619,female,上海,"劳斯莱斯, 玛莎拉蒂, 保时捷, 宾利, 宝马, 兰博基尼, 法拉利, 奔驰, 雷克萨斯, 奥迪"
3,扣_扣_扣扣篮,https://weibo.com/szeyuen1008,male,广东,"英语, 篮球, 爱狗, 衛蘭, 周傑倫, 保时捷, 凯迪拉克, 别克, 雷克萨斯, 起亚"
4,木个兰,https://weibo.com/u/2464870783,male,海外,"音乐, 雷克萨斯, 婚庆, 淑女, 不抽烟"
...,...,...,...,...,...
315,无锡宝星汽车贸易有限公司,https://weibo.com/u/2733514392,female,江苏,"雷克萨斯, 欧宝, 保时捷专卖"
316,何政一,https://weibo.com/u/2823617780,male,北京,"丰田金融, 雷克萨斯金融, 重点经销商集团, 汽车"
317,三亚中升LEXUS,https://weibo.com/u/7399107992,male,海南,雷克萨斯
318,板栗大嘴滚滚红尘w_181,https://weibo.com/u/1982424772,male,澳门,"暗示, 卡雅利沙的卡门, 雷克萨斯, 寂寞先生, 恋风恋歌, 地上最强新娘, 哈利波特"


In [225]:
data4 = find_bloggers2('0402_85e736@163.com', "cpy919992", "宝马")
df4 = dict_to_df2(data4)
df4
df4.to_csv('宝马_o.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag
0,VivianLin林靜文,https://weibo.com/jokerbabe,female,海外,"寶馬mm, 林靜文, 寶馬妹妹, 紋身, 刺青, vivian"
1,全球豪车馆,https://weibo.com/u/2335798650,male,云南,"汽车, 豪车, 名车, 跑车, 全球豪车馆, 宝马, 奔驰, 奥迪, 兰博基尼, 法拉利"
2,零号ME,https://weibo.com/michael2046,male,广东,"兰博基尼, 法拉利, 保时捷, 深圳车友, 劳斯莱斯, 豪车保养, 改装, 奔驰宝马奥迪, ..."
3,清风舞藤,https://weibo.com/u/1601296495,female,其他,"汉服同袍, HH1妙手仁心, 卿萱, 南风一线天, 孖女, TVB控, 黄mandy, 婠陵..."
4,dpkurtdp,https://weibo.com/blindkurt,male,海外,"汽车, 改装, 高端车改装, kurtuning, 宝马改装, 奔驰改装, 保时捷改装, 汽车改装"
...,...,...,...,...,...
554,18岁的博容gothic,https://weibo.com/u/1778555632,female,江苏,"宝马, nba, 铁盒盒控, 电影, 小清新, 听歌, 90后"
555,芋不见你,https://weibo.com/baozunbmw,male,山东,"宝马, 上海宝马, 宝马X6, 宝马控, 宝马3系, 宝马X1, 宝马4系, 宝马5系, 宝..."
556,一匹奔跑的猪,https://weibo.com/jx19880113,male,山东,"汽车, 青州五中, 85后, 潍坊, 来, BMW, 山东工商学院, 宝马, 凯迪拉克"
557,梦忆然出彼岸花了吗,https://weibo.com/mengyiran0423,female,广东,"宅, 抽风, 宝马, 峰璇"


In [226]:
data5 = find_bloggers2('0402_85e736@163.com', "cpy919992", "奔驰")
df5 = dict_to_df2(data5)
df5
df5.to_csv('奔驰_o.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag
0,牛眼看车,https://weibo.com/yzliupeng,male,江苏,"尚有兴趣, 已没职业, 汽车, 扬州, 中国移动, 奔驰, 梅赛德斯"
1,全球豪车馆,https://weibo.com/u/2335798650,male,云南,"汽车, 豪车, 名车, 跑车, 全球豪车馆, 宝马, 奔驰, 奥迪, 兰博基尼, 法拉利"
2,我是车迷秀,https://weibo.com/u/2092513523,male,广东,"大众, 福特, 雪佛兰, 尼桑, 奔驰, 米其林, 邓禄普, 两桶油, 体育资讯"
3,零号ME,https://weibo.com/michael2046,male,广东,"兰博基尼, 法拉利, 保时捷, 深圳车友, 劳斯莱斯, 豪车保养, 改装, 奔驰宝马奥迪, ..."
4,dpkurtdp,https://weibo.com/blindkurt,male,海外,"汽车, 改装, 高端车改装, kurtuning, 宝马改装, 奔驰改装, 保时捷改装, 汽车改装"
...,...,...,...,...,...
555,MINI_WON,https://weibo.com/kimhou,male,海外,"温哥华, 留学生, 奔驰, 宝马, MINI, Countryman, 跑车, 超跑, sfu"
556,18岁的博实Mandy,https://weibo.com/u/1778538814,female,贵州,"奔驰, 90后, K歌, 戏迷, 追求自由, 爱香港, 学生"
557,d尧b,https://weibo.com/u/1908932721,male,北京,"汽车达人, 风险投资VC, 私募股权投资, 行业分析, 足球是我的最爱, 股票投资, cfa..."
558,18岁的华荣gorgeous,https://weibo.com/u/1778387674,female,浙江,"奔驰, 八卦, 爵士鼓, 宅, 听歌, 自由"


In [228]:
data6 = find_bloggers2('0402_85e736@163.com', "cpy919992", "沃尔沃")
df6 = dict_to_df2(data6)
df6
df6.to_csv('沃尔沃_o.csv',  encoding = 'utf_8_sig')

,name,uid,gender,region,tag
0,宁述勇,https://weibo.com/nsy999,male,北京,"音乐, 时尚, 电影, 读书, 旅游, 营销, 公关, 红酒, 摄影, 沃尔沃帆船赛"
1,吉工大的神,https://weibo.com/u/1421247082,male,上海,"传祺, VOLVO, LC200, 陆巡, 沃尔沃"
2,国际汽车榜,https://weibo.com/514387345,female,北京,"宝马, 奔驰, 兰博基尼, TOYOTA, 劳斯莱斯, 奥迪AUDI, 沃尔沃VOLVO, ..."
3,刘彦哥哥,https://weibo.com/u/2491537167,male,安徽,"旅游, 唱歌, 汽车, 沃尔沃, 安利, 忧郁, 狮子座, 美图, 天蝎座, 亲情"
4,天蝎蝴蝶ww,https://weibo.com/u/1855510057,female,江苏,"时尚, 电影, 80后, 热舞, 天蝎座, 美食, 沃尔沃, 大波浪, 张玉珊, 豹纹控"
...,...,...,...,...,...
424,参观枪手楼专用,https://weibo.com/u/2312802184,male,广西,"我去二零零零年, 雅酷, 地下交通战, 兰花传奇, 长安沃尔沃, 难以抗拒你容颜, 蔡妍"
425,只道泥,https://weibo.com/u/2198163472,male,上海,"考维特, 沃尔沃天籁, 倒转地球, 两两相望, 孙浩, 迪桑特"
426,董洮LAND,https://weibo.com/u/7382342644,male,其他,"F1, 沃尔沃, 斯堪尼亚, 揽胜, 物理"
427,VCEPS_239,https://weibo.com/vceps,female,江苏,"沃尔沃, 研发中心, 汽车, 电动汽车"


In [227]:
df5.to_csv('奔驰_o.csv',  encoding = 'utf_8_sig')

In [215]:
username, password, topic = '0402_85e736@163.com', "cpy919992", "凯迪拉克"
driver = webdriver.Firefox()
driver.get('http://weibo.com/login.php')
time.sleep(5)
driver.find_element_by_name('username').send_keys(username)
driver.find_element_by_name('password').send_keys(password)
time.sleep(10) # time to input validation code

driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/div/div/div/div[3]/div[6]').click()
time.sleep(5)


driver.get('https://s.weibo.com/')
driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/a[2]').click()
time.sleep(5)

driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').clear()
driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').send_keys(topic)
driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/button').click()
time.sleep(5)

elem = driver.find_element_by_link_text('标签').click()
time.sleep(2)
ele = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[1]/ul/li[2]/span')
ActionChains(driver).move_to_element(ele).perform()
time.sleep(1)
driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[2]/div[1]/ul/li[2]/span/ul/li[4]/a').click()
time.sleep(5)

data = {}
for j in range(1, 50):
    try:
        s = driver.find_element_by_class_name('card.card-no-result.s-pt20b40')
        driver.refresh()
        time.sleep(5)
    except:
        time.sleep(1)

    for i in range(len(driver.find_elements_by_xpath('/html/body/div[1]/div[3]/div[1]/div[3]/*'))):

        table = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div[1]/div[3]/div['+str(i+1)+']/div[2]')

        if table.find_element_by_xpath('p[3]').text[:2] == '关注':
            continue

        name = table.find_element_by_xpath('div/a[1]').text    
        uid = table.find_element_by_class_name('name').get_attribute('href')
        gender = table.find_element_by_xpath('p[1]/i').get_attribute('class')
        gender = re.findall('icon-sex-(.*)', gender)[0]
        region = table.find_element_by_xpath('p[1]').text.split(' ')[0]
        tags = []
        for ele in table.find_elements_by_xpath('p'):
            if ele.text.startswith('标签'):
                tag_lst = ele.find_elements_by_xpath('a')
                tags = [tag.text for tag in tag_lst]

        data[len(data)] = {'name':name, 'uid':uid, 'gender':gender,'region':region,
                           'tag': tags}
    time.sleep(5)

    if j != 50:
        driver.find_element_by_class_name('next').click()
    time.sleep(4)

NoSuchElementException: Message: Unable to locate element: /html/body/div[1]/div[3]/div[1]/div[3]/div[1]/div[2]


In [199]:
data

{0: {'name': '潮流风景线',
  'uid': 'https://weibo.com/u/2700753641',
  'gender': 'male',
  'region': '河南',
  'tag': ['闺女徐贤要幸福', '罗塘人家', '爱情小白', '凯迪拉克SRX']},
 1: {'name': '扣_扣_扣扣篮',
  'uid': 'https://weibo.com/szeyuen1008',
  'gender': 'male',
  'region': '广东',
  'tag': ['英语', '篮球', '爱狗', '衛蘭', '周傑倫', '保时捷', '凯迪拉克', '别克', '雷克萨斯', '起亚']},
 2: {'name': '凯迪拉克长春宏明品鉴中心',
  'uid': 'https://weibo.com/726260820',
  'gender': 'male',
  'region': '吉林',
  'tag': ['凯迪拉克', '促销', '管理', '销售', '营销', '市场', '百年历史', '美国品牌', '豪华车', '汽车']},
 3: {'name': '北巷草猫',
  'uid': 'https://weibo.com/libamao',
  'gender': 'male',
  'region': '江苏',
  'tag': ['IT数码', '军事', '旅游', '新闻资讯', '汽车', '打球', '凯迪拉克']},
 4: {'name': '凯迪拉克大本营',
  'uid': 'https://weibo.com/u/2375199402',
  'gender': 'male',
  'region': '广东',
  'tag': ['凯迪拉克',
   'CTS',
   '轿跑',
   '钻石切割',
   'SLS',
   '凯雷德',
   '豪华车',
   'SRX',
   '汽车',
   '赛威']},
 5: {'name': '湖南凯迪拉克车友会',
  'uid': 'https://weibo.com/hnkdlk',
  'gender': 'male',
  'region': '湖南',
  'tag':

In [177]:
table.find_element_by_xpath('p[4]').text.startswith('标签')

True

In [178]:
elem = table.find_element_by_xpath('p[4]')

In [179]:
elem.find_elements_by_xpath('a')

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="913fd15a-aa3b-430b-9fa3-5488830be698", element="9f106cb2-c3be-4718-8358-4406e606935f")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="913fd15a-aa3b-430b-9fa3-5488830be698", element="fd75667d-7127-402e-92a9-454e43967ac5")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="913fd15a-aa3b-430b-9fa3-5488830be698", element="c3b4092e-aa87-4644-ac3e-eafc91ddbe22")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="913fd15a-aa3b-430b-9fa3-5488830be698", element="aad358b9-78f7-4abd-b0c3-340b1b51948e")>]

In [159]:
table.find_elements_by_xpath('p')

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="913fd15a-aa3b-430b-9fa3-5488830be698", element="c5ef1e61-7943-4331-806a-a821608883d9")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="913fd15a-aa3b-430b-9fa3-5488830be698", element="06fb8c19-b6ee-452f-9264-850d71b05fc3")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="913fd15a-aa3b-430b-9fa3-5488830be698", element="a9cdc202-daa6-4b98-969d-a3293ee1f7d0")>,
 <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="913fd15a-aa3b-430b-9fa3-5488830be698", element="5a3b5582-5f67-4a5d-bef5-e52a224df3c2")>]

In [108]:
if len(table.find_elements_by_xpath('p[2]/*')) == 0:
    tag = table.find_element_by_xpath('p[2]').text.split(' ')
    follows = table.find_element_by_xpath('p[3]/span/a').text
    fans = table.find_element_by_xpath('p[3]/span[2]/a').text
    posts = table.find_element_by_xpath('p[3]/span[3]/a').text
else:
    tag = []
    follows = table.find_element_by_xpath('p[2]/span/a').text
    fans = table.find_element_by_xpath('p[2]/span[2]/a').text
    posts = table.find_element_by_xpath('p[2]/span[3]/a').text

WebDriverException: Message: Failed to decode response from marionette


In [100]:
name = table.find_element_by_xpath('div/a[1]').text    
uid = table.find_element_by_class_name('name').get_attribute('href')
uid = re.findall('[0-9]+', uid)[0]
gender = table.find_element_by_xpath('p[1]/i').get_attribute('class')
gender = re.findall('icon-sex-(.*)', gender)[0]
region = table.find_element_by_xpath('p[1]').text.split(' ')[0]

In [103]:
region

'河南'